#### Extract OSM extracts and GTFS extracts

* https://osmcode.org/osmium-tool/manual.html
* https://github.com/triply-at/gtfsutils 



In [ ]:
# Extract boundingbox
bbox = [(4.70317, 52.47143), (5.02670, 52.25626)]
osm_src = f"1-data/2-gh/osm/europe-latest.osm.pbf" # TODO: Replace by planet.osm.pbf
osm_out = f"1-data/2-gh/osm/amsterdam.osm.pbf"

!osmium extract -b {bbox[0][0]},{bbox[0][1]},{bbox[1][0]},{bbox[1][1]} {osm_src} -o {osm_out} # --overwrite
!osmium fileinfo {osm_out}

In [ ]:
gtfs_in   =  "1-data/2-gh/gtfs/gtfs-nl.zip"
!gtfsutils bounds 1-data/2-gh/gtfs/gtfs-nl.zip

In [ ]:
gtfs_out  = "1-data/2-gh/gtfs/amsterdam.gtfs.zip"
gtfs_bbox = f' "[{bbox[0][0]},{bbox[0][1]},{bbox[1][0]},{bbox[1][1]}]"'

# Change parameter -o from intersects to within
!gtfsutils filter -t shapes -o within --overwrite -v {gtfs_in} {gtfs_out} {gtfs_bbox}

In [10]:
import shapely.geometry
import geopandas as gpd
import matplotlib.pyplot as plt

import gtfsutils
import gtfsutils.filter
import gtfsutils.routes

print(f"gtfsutils {gtfsutils.__version__}")


gtfsutils 0.0.5


In [ ]:
df_dict = gtfsutils.load_gtfs(gtfs_in)


### Create GraphHopper

In [ ]:
!docker run -d -v ${PWD}/1-data:/1-data --entrypoint /bin/bash israelhikingmap/graphhopper -c "java -jar *.jar server 1-data/2-gh/config-duttv2.yml"

In [ ]:
!docker run -v ${PWD}/1-data:/1-data --entrypoint /bin/bash israelhikingmap/graphhopper -c "java -Xmx10g -Xms10g *.jar server /1-data/2-gh/config-duttv2.yml"

In [9]:
!java -Xmx8g -Xms8g -jar graphhopper/web/target/graphhopper-web-*.jar server 1-data/2-gh/config-duttv2.yml